In [1]:
from vispy import gloo
from vispy import app
import numpy as np
from vispy.gloo import *
from vispy.gloo.context import FakeCanvas

In [2]:
shape = (600,800)
_rendertex = gloo.Texture2D((shape + (3,)))
fbo =  gloo.FrameBuffer(_rendertex, gloo.RenderBuffer(shape))
_ =  gloo.FrameBuffer(_rendertex, gloo.RenderBuffer(shape))

In [3]:
fake_canvas = FakeCanvas()

In [4]:
gloo.set_state(clear_color='white', blend=True,
                       blend_func=('src_alpha', 'one_minus_src_alpha'))

In [5]:
import vispy.io as io

from vispy import gloo
from vispy import app
import numpy as np
# from vispy.gloo import GLContext

VERT_SHADER = """
attribute vec2  a_position;
attribute vec3  a_color;
attribute float a_size;

varying vec4 v_fg_color;
varying vec4 v_bg_color;
varying float v_radius;
varying float v_linewidth;
varying float v_antialias;

void main (void) {
    v_radius = a_size;
    v_linewidth = 1.0;
    v_antialias = 1.0;
    v_fg_color  = vec4(0.0,0.0,0.0,0.5);
    v_bg_color  = vec4(a_color,    1.0);

    gl_Position = vec4(a_position, 0.0, 1.0);
    gl_PointSize = 2.0*(v_radius + v_linewidth + 1.5*v_antialias);
}
"""

FRAG_SHADER = """
#version 120

varying vec4 v_fg_color;
varying vec4 v_bg_color;
varying float v_radius;
varying float v_linewidth;
varying float v_antialias;
void main()
{
    float size = 2.0*(v_radius + v_linewidth + 1.5*v_antialias);
    float t = v_linewidth/2.0-v_antialias;
    float r = length((gl_PointCoord.xy - vec2(0.5,0.5))*size);
    float d = abs(r - v_radius) - t;
    if( d < 0.0 )
        gl_FragColor = v_fg_color;
    else
    {
        float alpha = d/v_antialias;
        alpha = exp(-alpha*alpha);
        if (r > v_radius)
            gl_FragColor = vec4(v_fg_color.rgb, alpha*v_fg_color.a);
        else
            gl_FragColor = mix(v_bg_color, v_fg_color, alpha);
    }
}
"""

In [6]:
# Create vertices
n = 10000
ps=1
v_position = 0.25 * np.random.randn(n, 2).astype(np.float32)
v_color = np.random.uniform(0, 1, (n, 3)).astype(np.float32)
v_size = np.random.uniform(2*ps, 12*ps, (n, 1)).astype(np.float32)

program = gloo.Program(VERT_SHADER, FRAG_SHADER)

In [7]:
# Set uniform and attribute
program['a_color'] = gloo.VertexBuffer(v_color)
program['a_position'] = gloo.VertexBuffer(v_position)
program['a_size'] = gloo.VertexBuffer(v_size)

In [8]:
# with fbo:
fbo.activate()
gloo.clear(color=True, depth=True)
# gloo.set_viewport(0, 0, *self.physical_size)
program.draw('points')

IndexError: list index out of range